In [1]:
!pip install accelerate

In [2]:
import os
import gc
import sys
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from accelerate import Accelerator
from transformers import (AutoModel,AutoTokenizer,AdamW,
                          AutoModelForNextSentencePrediction,
                          AutoConfig,get_cosine_schedule_with_warmup)

from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [3]:
config = {
    'lr': 1.1e-5,
    'wd':1e-2,
    'batch_size':256,
    'num_workers':4,
    'max_len':6,
    'epochs':10,
    'nfolds':5,
    'seed':1000,
    'model_path':'bert-base-uncased'
}

for i in range(config['nfolds']):
    os.makedirs(f'model{i}',exist_ok=True)

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])

## Making Predictions

In [21]:
# test_data = train_data.query("Fold == 0")
# test_data = train_data
test_data = pd.read_csv('../input/google-ads-rur-data/collagen_mask_data.csv')
# keyword1 = test_data['keyword1'].to_list()
# test_data['keyword1'] = test_data['keyword2']
# test_data['keyword2'] = keyword1

In [22]:
test_data.head()

,keyword1,keyword2
0,collagen mask,collagen mask
1,collagen mask,collagen face mask
2,collagen mask,collagen lips mask
3,collagen mask,collagen lip plumping mask
4,collagen mask,crystal collagen gold powder eye mask


In [23]:
class GadsTestDataset(Dataset):
    def __init__(self,df,tokenizer):
        self.keywords1 = df['keyword1'].to_numpy()
        self.keywords2 = df['keyword2'].to_numpy()
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.keywords1[idx],
                                text_pair=self.keywords2[idx],
                                return_tensors='pt',
                                max_length=config['max_len'],
                                padding='max_length',truncation=True)
        return encode
    
    def __len__(self):
        return len(self.keywords1)

In [24]:
def get_prediction(df,path,model_path,device='cuda'):        
    model = AutoModelForNextSentencePrediction.from_pretrained(config['model_path'])
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    predictions = list()
    
    for f in range(config['nfolds']):
        model.load_state_dict(torch.load(path.format(f),map_location=device))
        model.to(device)
        model.eval()

        test_ds = GadsTestDataset(df,tokenizer)
        test_dl = DataLoader(test_ds,
                            batch_size = config["batch_size"],
                            shuffle=False,
                            drop_last=False,
                            num_workers = 4,
                            pin_memory=True)

        with torch.no_grad():
            pred = list()
            for i, (inputs) in tqdm(enumerate(test_dl)):
                inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
                outputs = model(**inputs)
                outputs = outputs['logits'].cpu().detach().numpy().tolist()
                pred.extend(outputs)
            predictions.append(pred)
            
    torch.cuda.empty_cache()
    final_predictions = np.mean(predictions,axis=0)
    return np.mean(predictions,axis=0)

In [25]:
model_paths = [
    ['../input/rur-roberta-models/model{0}/model{0}.bin',config['model_path']],
]

In [26]:
predictions = np.zeros((test_data.shape[0],2))
for path,model_path in model_paths:
    predictions += get_prediction(test_data,path,model_path)/len(model_paths)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2it [00:00,  4.41it/s]
2it [00:00,  4.57it/s]
2it [00:00,  4.60it/s]
2it [00:00,  4.59it/s]
2it [00:00,  4.51it/s]


In [27]:
predictions = np.argmax(predictions,axis=1)

In [28]:
test_data['predictions'] = predictions

In [29]:
test_data.to_csv('collagen_mask_prediction.csv',index=False)

In [14]:
# accuracy_score(test_data['target'],test_data['predictions'])

In [30]:
relevant_data = test_data.query('predictions==1')
relevant_data.to_csv('collagen_mask_relevant.csv',index=False)

In [34]:
test_data['predictions'].value_counts()

0    257
1    248
Name: predictions, dtype: int64

In [31]:
test_data.head(50)

,keyword1,keyword2,predictions
0,collagen mask,collagen mask,0
1,collagen mask,collagen face mask,0
2,collagen mask,collagen lips mask,0
3,collagen mask,collagen lip plumping mask,0
4,collagen mask,crystal collagen gold powder eye mask,1
5,collagen mask,collagen eye mask,0
6,collagen mask,gold collagen lip mask,0
7,collagen mask,collagen sheet mask,0
8,collagen mask,mario badescu super collagen mask,1
9,collagen mask,diy collagen mask,0


In [32]:
relevant_data.head(50)

,keyword1,keyword2,predictions
4,collagen mask,crystal collagen gold powder eye mask,1
8,collagen mask,mario badescu super collagen mask,1
17,collagen mask,soo ae collagen mask review,1
18,collagen mask,under eye collagen mask,1
19,collagen mask,best collagen face mask,1
22,collagen mask,gold bio collagen face mask,1
23,collagen mask,crystal collagen gold powder facial mask,1
24,collagen mask,best collagen mask,1
25,collagen mask,dellure collagen mask,1
26,collagen mask,bio miracle collagen mask,1
